## Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import warnings
from google.colab import files
import pickle
import os

!pip install scipy
import scipy.stats as stats

!pip install papermill
!pip install nbconvert
!pip install nbformat
!pip install IPython

import papermill as pm
import nbformat
from nbconvert import HTMLExporter
from IPython.display import HTML, display

import json
from google.colab import drive

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import warnings
from google.colab import files
import pickle
import os
from scipy import stats
from scipy.stats import zscore

!pip install missingno
import missingno as msno

!pip install fancyimpute
import fancyimpute

warnings.filterwarnings("ignore")
%matplotlib inline

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 22.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 4.6 MB/s eta 0:00:00
  Created wheel for fancyimpute: filename=fancyimpute-0.7.0-py3-none-any.whl size=29879 sha256=99d70dc38d7cf9c0b7e9be302a1814286606680b55ef2a01f7d201b12b749de6
  Stored in directory: /root/.cache/pip/wheels/df/20/91/e4850b9a31cf660c1bc95515d3bcbc8010e869e5de6d5baf07
  Created wheel for knnimpute: filename=knnimpute-0.1.0-py3-none-any.whl size=11331 sha256=8b1b2a086d83caf26dc38feca908fe6620ea0c8c672a58f4f97b89d898494529
  Stored in directory: /root/.cache/pip/wheels/89/11/61/aa5a3167bfff49218cd043a73a83034b9eadd858b0059521be
Successfully built fancyimpute knnimpute


# Bootstrap

In [2]:
import os
from dotenv import load_dotenv

# Find and load the .env file from the current or parent directories
load_dotenv()

# Access the environment variable using os.getenv()
PROJECT_PATH = os.getenv('PROJECT_PATH')
CONFIG_FILE = f"{PROJECT_PATH}/src/config.json"

In [3]:
drive.mount('/content/drive')

with open(CONFIG_FILE, 'r') as f:
    project_config = json.load(f)
    project_config.pop('_comment', None)
    project_config.pop('_note', None)
    f.close()

Mounted at /content/drive


In [4]:
np.random.seed(31071967)

#Run project notebook N-1

In [5]:
if project_config['chain_notebooks'] == '1':

  input_file = f"{PROJECT_PATH}{project_config['notebooks_directory']}{project_config['notebook3']}"
  output_file = f"{PROJECT_PATH}{project_config['output_directory']}{project_config['output3']}"

  # --- Execute the proviuse notebook with parameters ---
  pm.execute_notebook(
      input_path = input_file,
      output_path = output_file,
      log_output=False,  # don't print logs while running
      progress_bar=True
  )

  # --- Convert the executed notebook to HTML ---
  nb = nbformat.read(output_file, as_version=4)
  html_exporter = HTMLExporter()
  html_exporter.template_name = "lab"  # modern look; alternatives: 'classic', 'basic'
  body, _ = html_exporter.from_notebook_node(nb)

  # --- Display the HTML result inline ---
  display(HTML(body))

Output hidden; open in https://colab.research.google.com to view.

# Load pickle

In [6]:
pickle_file      = PROJECT_PATH + project_config['pickles_directory'] + project_config['pickle3']
test_pickle_file = PROJECT_PATH + project_config['pickles_directory'] + project_config['pickle3_test']

In [7]:
# Read pickle into DataFrame
df = pd.read_pickle(pickle_file)
display(df.head(1))

if project_config['split_df'] == '1':
  df_test = pd.read_pickle(test_pickle_file)
  display(df_test.tail(1))

,track_id,track_artist,track_popularity,track_album_id,playlist_id,danceability,energy,key,loudness,mode,...,release_decade_mean_popularity,release_month_mean_popularity,release_year_mean_popularity,playlist_genre_grouped,track_artist_mean_popularity,track_album_id_mean_popularity,playlist_id_mean_popularity,mode_mean_popularity,key_mean_popularity,playlist_genre_grouped_mean_popularity
26210,6GnhWMhgJb7uyiiPEiEkDA,Mac Miller,77,2Tyx5dLhHYkx6zeAdVaTzN,2ujrP2rPxu8ao7wNeG1ZOY,0.843,0.435,G♯/A♭,-8.442,Major,...,43.817673,44.145998,35.485437,Soul & Big Room EDM,67.933333,77.0,39.895522,42.769249,44.744681,32.370314


,track_id,track_artist,track_popularity,track_album_id,playlist_id,danceability,energy,key,loudness,mode,...,release_decade_mean_popularity,release_month_mean_popularity,release_year_mean_popularity,playlist_genre_grouped,track_artist_mean_popularity,track_album_id_mean_popularity,playlist_id_mean_popularity,mode_mean_popularity,key_mean_popularity,playlist_genre_grouped_mean_popularity
20687,4vp2J1l5RD4gMZwGFLfRAu,The Black Eyed Peas,69,36fdxiOzdlmsrHgGcfvqUJ,6a66cg3HcsjYkisYyQcov6,0.743,0.766,C,-6.375,Major,...,33.36481,38.1139,28.538012,Electro/Latin Pop Fusion,56.625,69.0,58.826087,42.573333,41.820202,43.353232


In [8]:
# types of cols
small_cat_cols     = ['mode_mean_popularity','key_mean_popularity', 'playlist_genre_grouped_mean_popularity','release_decade_mean_popularity','release_month_mean_popularity']
small_cat_symboles = ['mode','key','playlist_genre_grouped','release_decade','release_month', 'release_year']
large_cat_cols     = ['track_artist_mean_popularity','track_album_id_mean_popularity','playlist_id_mean_popularity']
cont_cols          = ['acousticness', 'danceability','duration_ms','energy', 'liveness', 'loudness', 'speechiness', 'tempo','valence']
y_col              = 'track_popularity'
X_cols             = large_cat_cols + small_cat_cols + cont_cols
MERGE_ON_COL       = 'track_id'

In [9]:
def pickle_col(df, col='all', drop_col=False, include_merge_ID=True, pickle_name=""):

  import pickle

  if pickle_name == "":
    file_name = f"{PROJECT_PATH}{project_config['pickles_directory']}{col}.pkl"
  else:
    file_name = f"{PROJECT_PATH}{project_config['pickles_directory']}{pickle_name}.pkl"

  with open(file_name, 'wb') as f:

    if col =='all':
      pickle.dump(df, f)

    elif col in df.columns: # in case we aready droped the col before

      # track_id for a later merge, if we need.
      #and y_col so can can invetigate the pickel later indepandantly from the main df
      pickle.dump(df[[MERGE_ON_COL, col, y_col]], f)

    f.close()

    if drop_col == True:
      df.drop(col, axis=1, inplace=True, errors='ignore')

  if project_config['split_df'] == '1':
    with open(file_name+".test.pkl", 'wb') as f:

      if col =='all':
        pickle.dump(df_test, f)

      elif col in df_test.columns: # in case we aready droped the col before
        pickle.dump(df_test[[MERGE_ON_COL, col, y_col]], f)

      f.close()

    if drop_col == True:
      df_test.drop(col, axis=1, inplace=True, errors='ignore')

  return df

# Target Encoding


In [10]:
# artist_id/name --> artist_mean_popularity
# track_album_id --> track_album_mean_popularity
# track_playlist_id --> playlist_id_mean_popularity

# mode --> mode_mean_popularity
# key --> key_mean_popularity
# genre --> genre_mean_popularity

# release_date --> release_month_mean_popularity, release_year_mean_popularity, release_decade_mean_popularity

#Feature selection

In [11]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVR
from sklearn.linear_model import Ridge

In [12]:
# Fit models and determine if a feature is selected (1) or not (0)
lasso = Lasso(alpha=5).fit(df[X_cols], df[y_col])
lasso_selected = (np.abs(lasso.coef_) > 0).astype(int)

In [13]:
# Fit Ridge model
ridge = Ridge(alpha=5).fit(df[X_cols], df[y_col])
ridge_selected = (np.abs(ridge.coef_) > 0).astype(int)

In [14]:
gb = GradientBoostingRegressor().fit(df[X_cols], df[y_col])
gb_selected = (gb.feature_importances_ > 0).astype(int)

In [15]:
rf = RandomForestRegressor().fit(df[X_cols], df[y_col])
rf_selected = (rf.feature_importances_ > 0).astype(int)

In [16]:
# Create a DataFrame to store results
selection_df = pd.DataFrame({
    'Feature': df[X_cols].columns,
    'Lasso': lasso_selected,
    'GradientBoost': gb_selected,
    'RandomForest': rf_selected,
    'Ridge': ridge_selected
})

# Sum the number of selections for each feature
selection_df['Sum'] = selection_df[['Lasso', 'GradientBoost', 'RandomForest','Ridge']].sum(axis=1)

# Output the results
display(selection_df)

,Feature,Lasso,GradientBoost,RandomForest,Ridge,Sum
0,track_artist_mean_popularity,0,1,1,1,3
1,track_album_id_mean_popularity,1,1,1,1,4
2,playlist_id_mean_popularity,1,1,1,1,4
3,mode_mean_popularity,0,1,1,1,3
4,key_mean_popularity,0,1,1,1,3
5,playlist_genre_grouped_mean_popularity,0,1,1,1,3
6,release_decade_mean_popularity,0,1,1,1,3
7,release_month_mean_popularity,0,0,1,1,2
8,acousticness,0,1,1,1,3
9,danceability,0,1,1,1,3


In [17]:
for feature in selection_df[selection_df['Sum'] < 3]['Feature']:
  print(f"dropping features with no majority of recomendations: {feature}")
  pickle_col(df, col=feature, drop_col=True)

dropping features with no majority of recomendations: release_month_mean_popularity


In [18]:
from google.colab import files
import os

with open(f"{PROJECT_PATH}{project_config['pickles_directory']}{project_config['pickle4']}", 'wb') as f:
  pickle.dump(df, f)
  f.close()

if project_config['split_df'] == '1':
  with open(f"{PROJECT_PATH}{project_config['pickles_directory']}{project_config['pickle4_test']}", 'wb') as f:
    pickle.dump(df_test, f)
    f.close()